In [1]:
from qh_circuit import *
from qh_gates import *
from mps_simu import MPS_Simulator
import qiskit
from qiskit import Aer

import matplotlib.pyplot as plt

In [2]:
def fidelity(sv1: np.ndarray, sv2: np.ndarray):
    sv1_normed = sv1 / np.linalg.norm(sv1) ** 2
    sv2_normed = sv2 / np.linalg.norm(sv2) ** 2
    return np.abs(np.inner(sv1_normed, sv2_normed.conj())) ** 2

In [3]:
def plot_state(amp):
    plt.figure()
    p = np.abs(amp) ** 2
    n = int(np.log2(len(amp)))
    label = [(r"{:0" + f"{n}" + r"b}").format(i) for i in range(len(amp))]
    plt.bar(label, p)
    plt.xticks(rotation=70)
    plt.title(f"{p.sum()}")
    plt.show()
    plt.close()

In [4]:
supported_gates = list(gate_dict.keys())
print(f"Supported gates: {supported_gates}")

Supported gates: ['X', 'Y', 'Z', 'H', 'S', 'T', 'P', 'RX', 'RY', 'RZ', 'CX', 'RXX', 'RYY', 'RZZ', 'MEASURE']


In [5]:
qc = QHCircuit(10)
for gate in supported_gates:
    try:
        dir(qc).index(gate.lower())
    except:
        print(f"Warning: Method {gate} is not defined in QHCircuit")

In [6]:
def check_one_qubit_gates(nqubits=4, ngates=3, round=10):
    failed = []
    for _ in range(round):
        mps_qc = QHCircuit(nqubits)
        qiskit_qc = qiskit.QuantumCircuit(nqubits)

        for _ in range(ngates):
            gate = np.random.randint(0, 10)
            qubit = np.random.randint(0, nqubits)
            if gate == 0: # x
                mps_qc.x(qubit)
                qiskit_qc.x(qubit)
            elif gate == 1: # y
                mps_qc.y(qubit)
                qiskit_qc.y(qubit)
            elif gate == 2: # z
                mps_qc.z(qubit)
                qiskit_qc.z(qubit)
            elif gate == 3: # h
                mps_qc.h(qubit)
                qiskit_qc.h(qubit)
            elif gate == 4: # s
                mps_qc.s(qubit)
                qiskit_qc.s(qubit)
            elif gate == 5: # t
                mps_qc.t(qubit)
                qiskit_qc.t(qubit)
            elif gate == 6: # p
                para = np.random.rand()
                mps_qc.p(qubit, para)
                qiskit_qc.p(para, qubit)
            elif gate == 7: # rx
                para = np.random.rand()
                mps_qc.rx(qubit, para)
                qiskit_qc.rx(para, qubit)            
            elif gate == 8: # ry
                para = np.random.rand()
                mps_qc.ry(qubit, para)
                qiskit_qc.ry(para, qubit)            
            elif gate == 9: # rz
                para = np.random.rand()
                mps_qc.rz(qubit, para)
                qiskit_qc.rz(para, qubit)

        mps_simulator = MPS_Simulator(mps_qc, xi=1e-8, max_chi=2**ngates)
        qiskit_simulator = Aer.get_backend('statevector_simulator')

        mps_simulator.simulate()
        mps_sv = mps_simulator.get_all_amplitudes()
        qiskit_sv = qiskit_simulator.run(qiskit_qc).result().get_statevector().data

        fid = fidelity(mps_sv, qiskit_sv)
        if fid < 1 - 1e-3:
            failed.append(mps_qc)
            print(f"Failed! Fidelity = {fid}")

    if len(failed) == 0:
        print(f"All successful")
    else:
        return failed 


In [7]:
check_one_qubit_gates(nqubits=4, ngates=100, round=10)

All successful


In [8]:
def check_cnot(nqubits=4, ngates=3, round=10):
    failed = []
    for _ in range(round):
        mps_qc = QHCircuit(nqubits)
        qiskit_qc = qiskit.QuantumCircuit(nqubits)

        for _ in range(ngates):
            gate = np.random.randint(0, 6)
            qubit = np.random.randint(0, nqubits-1)
            qubit2 = qubit+1 
            if np.random.rand() > 0.5:
                qubit, qubit2 = qubit2, qubit
            if gate == 0: # h
                mps_qc.h(qubit)
                qiskit_qc.h(qubit)
            elif gate == 1: # p
                para = np.random.rand()
                mps_qc.p(qubit, para)
                qiskit_qc.p(para, qubit)
            elif gate == 2: # rx
                para = np.random.rand()
                mps_qc.rx(qubit, para)
                qiskit_qc.rx(para, qubit)            
            elif gate == 3: # ry
                para = np.random.rand()
                mps_qc.ry(qubit, para)
                qiskit_qc.ry(para, qubit)            
            elif gate == 4: # rz
                para = np.random.rand()
                mps_qc.rz(qubit, para)
                qiskit_qc.rz(para, qubit)
            elif gate == 5: # cnot
                mps_qc.cnot(qubit, qubit2)
                qiskit_qc.cnot(qubit, qubit2)


        mps_simulator = MPS_Simulator(mps_qc, xi=1e-8, max_chi=2**ngates)
        qiskit_simulator = Aer.get_backend('statevector_simulator')
    
        mps_simulator.simulate()
        mps_sv = mps_simulator.get_all_amplitudes()
        qiskit_sv = qiskit_simulator.run(qiskit_qc).result().get_statevector().data
        
        fid = fidelity(mps_sv, qiskit_sv)
        if fid < 1 - 1e-3:
            failed.append(mps_qc)
            print(f"Failed! Fidelity = {fid}")

    if len(failed) == 0:
        print(f"All successful")
    else:
        return failed 


In [9]:
failed = check_cnot(nqubits=4, ngates=10, round=10)

All successful


In [10]:
def check_two_qubit_gates(nqubits=4, ngates=3, round=10):
    failed = []
    for _ in range(round):
        mps_qc = QHCircuit(nqubits)
        qiskit_qc = qiskit.QuantumCircuit(nqubits)

        for _ in range(ngates):
            gate = np.random.randint(0, 9)
            qubit = np.random.randint(0, nqubits-1)
            qubit2 = qubit+1 
            if np.random.rand() > 0.5:
                qubit, qubit2 = qubit2, qubit         
                   
            if gate == 0: # h
                mps_qc.h(qubit)
                qiskit_qc.h(qubit)
            elif gate == 1: # p
                para = np.random.rand()
                mps_qc.p(qubit, para)
                qiskit_qc.p(para, qubit)
            elif gate == 2: # rx
                para = np.random.rand()
                mps_qc.rx(qubit, para)
                qiskit_qc.rx(para, qubit)            
            elif gate == 3: # ry
                para = np.random.rand()
                mps_qc.ry(qubit, para)
                qiskit_qc.ry(para, qubit)            
            elif gate == 4: # rz
                para = np.random.rand()
                mps_qc.rz(qubit, para)
                qiskit_qc.rz(para, qubit)
            elif gate == 5: # cnot
                mps_qc.cnot(qubit, qubit2)
                qiskit_qc.cnot(qubit, qubit2)
            elif gate == 6: # rxx
                para = np.random.rand()
                mps_qc.rxx(qubit, qubit2, para)
                qiskit_qc.rxx(para, qubit, qubit2)
            elif gate == 7: # ryy
                para = np.random.rand()
                mps_qc.ryy(qubit, qubit2, para)
                qiskit_qc.ryy(para, qubit, qubit2)
            elif gate == 8: # rzz
                para = np.random.rand()
                mps_qc.rzz(qubit, qubit2, para)
                qiskit_qc.rzz(para, qubit, qubit2)

        mps_simulator = MPS_Simulator(mps_qc, xi=1e-8, max_chi=2**ngates)
        
        qiskit_simulator = Aer.get_backend('statevector_simulator')
    
        mps_simulator.simulate()
        mps_sv = mps_simulator.get_all_amplitudes()
        qiskit_sv = qiskit_simulator.run(qiskit_qc).result().get_statevector().data
        
        fid = fidelity(mps_sv, qiskit_sv)
        if fid < 1 - 1e-3:
            failed.append(mps_qc)
            print(f"Failed! Fidelity = {fid}")

    if len(failed) == 0:
        print(f"All successful")
    else:
        return failed 


In [11]:
failed = check_two_qubit_gates(nqubits=7, ngates=30, round=10)

All successful


In [12]:
def check_measurement(nqubits=4, ngates=3, round=10):
    failed = []
    for _ in range(round):
        mps_qc = QHCircuit(nqubits)
        qiskit_qc = qiskit.QuantumCircuit(nqubits)

        for _ in range(ngates):
            gate = np.random.randint(0, 10)
            qubit = np.random.randint(0, nqubits-1)
            qubit2 = qubit+1 
            if np.random.rand() > 0.5:
                qubit, qubit2 = qubit2, qubit         
                   
            if gate == 0: # h
                mps_qc.h(qubit)
                qiskit_qc.h(qubit)
            elif gate == 1: # p
                para = np.random.rand()
                mps_qc.p(qubit, para)
                qiskit_qc.p(para, qubit)
            elif gate == 2: # rx
                para = np.random.rand()
                mps_qc.rx(qubit, para)
                qiskit_qc.rx(para, qubit)            
            elif gate == 3: # ry
                para = np.random.rand()
                mps_qc.ry(qubit, para)
                qiskit_qc.ry(para, qubit)            
            elif gate == 4: # rz
                para = np.random.rand()
                mps_qc.rz(qubit, para)
                qiskit_qc.rz(para, qubit)
            elif gate == 5: # cnot
                mps_qc.cnot(qubit, qubit2)
                qiskit_qc.cnot(qubit, qubit2)
            elif gate == 6: # rxx
                para = np.random.rand()
                mps_qc.rxx(qubit, qubit2, para)
                qiskit_qc.rxx(para, qubit, qubit2)
            elif gate == 7: # ryy
                para = np.random.rand()
                mps_qc.ryy(qubit, qubit2, para)
                qiskit_qc.ryy(para, qubit, qubit2)
            elif gate == 8: # rzz
                para = np.random.rand()
                mps_qc.rzz(qubit, qubit2, para)
                qiskit_qc.rzz(para, qubit, qubit2)
            elif gate == 9: # measure
                if np.random.rand() < 0.5:
                    mps_qc.collapse(qubit, 0)
                    qiskit_qc.reset(qubit)
                else:
                    mps_qc.collapse(qubit, 1)
                    qiskit_qc.reset(qubit)
                    qiskit_qc.x(qubit)

        # print(mps_qc)
        mps_simulator = MPS_Simulator(mps_qc, xi=1e-8, max_chi=2**ngates)
        
        qiskit_simulator = Aer.get_backend('statevector_simulator')

        mps_simulator.initialize()
        mps_simulator.simulate()
        mps_sv = mps_simulator.get_all_amplitudes()
        qiskit_qc = qiskit.transpile(qiskit_qc, qiskit_simulator)
        qiskit_sv = qiskit_simulator.run(qiskit_qc).result().get_statevector().data
        
        fid = fidelity(mps_sv, qiskit_sv)
        if fid < 1 - 1e-3:
            failed.append(mps_qc)
            print(f"Failed! Fidelity = {fid}")
            print(mps_sv)
            print(qiskit_sv)

    if len(failed) == 0:
        print(f"All successful")
    else:
        return failed 


In [15]:
failed = check_measurement(nqubits=2, ngates=2, round=1000)

Failed! Fidelity = 0.0
[0.+0.j 1.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 0.-1.j]
Failed! Fidelity = 0.0
[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 0.+1.j]


In [16]:
print(failed[0])

RXX(0.439) on [1, 0]
COLLAPSE(1.000) on [0]
